In [18]:
import numpy as np
import matplotlib.pyplot as plt
from abc import ABC
import matplotlib.gridspec as gridspec
from datetime import date, datetime
%matplotlib inline

# Data Generation

In [19]:
def generate_data(T, K, mus, deltas, stochastic_data, adv = False):
  """
  Chooses whihc type of data to generate. If adv is given, it overrides stochastic_data
  """
  if stochastic_data and not adv:
    return generate_stoch_data(T, K, mus)
  elif not stochastic_data and not adv:
    return generate_non_stoch_data(T, K, deltas)
  else:
   return generate_adv_data(T, K)

### Stochastic Data

In [ ]:
def generate_stoch_data(T, K, mus):
  """
  generate loss sequences based on the given mus
  """
  data = np.random.rand(T, K)
  for i, mu in enumerate(mus):
    data[:,i] = np.array([data[:,i] < mu])
  return data

### Non Stochastic Data


In [ ]:
def generate_non_stoch_data(T, K, deltas):
  """
  directly generate non stochastic loss sequences based on the given mus
  """
  data = np.random.rand(T, K)
  flip = False
  ind = 0
  inv_deltas = deltas - np.max(deltas)

  for j in range(T):
    for i, (delta, inv_delta) in enumerate(zip(deltas, inv_deltas)):
      if flip:
        data[j,i] = int(data[j,i] < delta)
      else:
        data[j,i] = int(data[j,i] < (1 + inv_delta))
    
    if j > 1.6**ind:
      flip = bool(1-flip)
      ind+=1
      
  return data

### Adversarial Data
One example of adversarial data

In [ ]:
def generate_adv_data(T, K):
  """
  generates one type of adversarial data, starting with a unique optimal arm for
  the first rounds until the time horizon, after which that arm becomes the 
  unique bad arm
  """
  horizon = int(np.sqrt(K* T* np.log(K*T)))
  data = np.ones([T, K])
  data[:horizon, 0] = 0
  data[horizon:, 1:] = 0

  return data

# Interface for the Different Algorithms

## The Method class
Each method that we are testing has to contain the parameter label to work with the plotting function.

Distribution based algorithm is a class shared by all learners who compute a distribution at each round. This structure support the use of blocks.

Batched bandits has its own class as the BaSE algorithm does not choose arms to play by sampling from a distribution.

### Distribution Based algorithm
Common interface that can be used for algorithms that calculate a distribution at each round.
Both Tsallis-INF and EXP3 are implemented using this interface.

#### Block Construction
We consider both our adaptive sequence of blocks and a fixed sequence of block.
Our adaptive sequence only requires knowledge of the switching cost $\lambda$ and the number of arms.
The fixed sequence of blocks requires also the knowledge of the time horizon $T$.

In [ ]:
def bn(n):
  return 3/2 * lam * np.sqrt(n/K)

def make_blocks(T, K, N, lam, adaptive):
  '''
  block seq contains the first index of each block. 
  '''
  if adaptive:
    block_seq = np.zeros(N+1)
    bns = np.zeros(N+1)
    for n in range(1, N+1):
      bn = 3/2 * lam * np.sqrt(n/K)
      block_seq[n] = block_seq[n-1] + np.max([np.ceil(bn), 1])
      bns[n] = bn
    block_seq[-1] = T + 1 
    return block_seq, bns
  else:
    block_seq = np.zeros(N+1)
    bns = np.zeros(N+1)
    for n in range(1, N+1):
      block_seq[n] = int(n*T/N + 1)
      bns[n] = T/N
    block_seq[-1] = T + 1 
    return block_seq, bns


#### The Distribution based Class
This class can be used an adapted for any algorithm which computes distributions.


In [ ]:
class Distrib_based:
    def __init__(self, label, K, T, method = 'Tsallis', use_blocks = False, adaptive = True, switching_cost = 0):
      self.label = label
      self.x = -1
      self.algo = distrib_based_alg
      self.method = method
      self.set_blocks(use_blocks, K, T, switching_cost, adaptive)
      self.use_blocks = use_blocks
      self.T = T
      
    def set_blocks(self, use_blocks, K, T, switching_cost, adaptive):
      if use_blocks:
        N = T
        if switching_cost > 0:
          N = int(T**(2/3)*K**(1/3)/switching_cost**(2/3))
        blocks, bns = make_blocks(T, K, N, switching_cost, adaptive)
        last_block = np.argmin(blocks < T)
        self.blocks = blocks[:last_block+1]
        self.N = len(blocks)
        self.bns = bns
      else:
        self.blocks = range(T+1)
        self.N = T
        self.bns = np.ones(T+1)

    def one_step(self, n, K, L):
      if self.method == 'Tsallis':
        w = self.compute_Tsallis_distrib(n, K, L)
      else:
        w = self.compute_EXP3_distrib(n, K, L)
      return int(np.random.choice(K, 1, p = w)), w

    ### Tsallis Specific code
    def compute_Tsallis_distrib(self, n, K, L):
      if n == 1:
        self.x = -1
      eta = 1
      if self.use_blocks:
        eta = self.eta_Switching_Tsallis(n)
      else:
        eta = self.eta_MAB_Tsallis(n)
      stop = False
      i = 0
      x = self.x
      while not stop:
        w = 4*(eta*(L - x))**(-2)
        x = x - (np.sum(w)-1)/(eta*np.sum([wi**(3/2) for wi in w]))
        stop = np.abs(np.sum(w) - 1) < 10**(-8)
        i = i+1
        if i > 100:
          stop = True
        self.x = x
      return w

    def eta_Switching_Tsallis(self, n):
      return 2*np.sqrt(2)/(np.sqrt(n)*(self.bns[n] + 1))

    def eta_MAB_Tsallis(self, t):
      return 2/np.sqrt(t)

    ### EXP3 specific code
    def compute_EXP3_distrib(self, n, K, L):
      if self.use_blocks:
        eta = np.sqrt(self.N * np.log(K)/(self.T**2*K))
      else:
        eta = np.sqrt(np.log(K)/(n*K))

      w = np.exp(- eta* L)
      return w/np.sum(w)


In [ ]:
def distrib_based_alg(K, T, data, deltas, method, n_points, points_indices, verbose = True, adv = False):

  block_seq = method.blocks

  L = np.zeros(K)
  algo_data_reg = np.zeros(n_points)
  algo_switch_loss = np.zeros(n_points)
  action_losses = np.zeros(K)
  algo_total_loss = 0

  i = 0
  t = 1
  n = 1
  x = -1
  I_prev = -1
  cum_reg = 0
  cum_switch = 0
  ind = 0
  while i < T:
    t = i + 1
    I, w = method.one_step(n, K, L)

    while i < T and i < block_seq[n]:
      lossIt = data[i, I]
      unbiasedest = lossIt/w[I]
      L[I] = L[I] + unbiasedest

      cum_reg += deltas[I]
      action_losses += data[i]
      algo_total_loss += data[i, I]

      if i == points_indices[ind]:
        if adv:
          best_arm_loss = np.min(action_losses)
          algo_data_reg[ind] = algo_total_loss - best_arm_loss
        else:
          algo_data_reg[ind] = cum_reg
        algo_switch_loss[ind] = cum_switch
        ind += 1

      i = i + 1
    cum_switch += 1 - (I_prev == I)
    I_prev = I

    n = n+1
  
    if verbose and i%1000 == 0:
      print(i, n, I, w)

  if adv:
    best_arm_loss = np.min(action_losses)
    algo_data_reg[ind] = algo_total_loss - best_arm_loss
  else:
    algo_data_reg[ind] = cum_reg
  algo_switch_loss[ind] = cum_switch  
  return algo_data_reg, algo_switch_loss
    

### Batched Bandits
Based on the original implementation of
https://github.com/Mathegineer/batched-bandit

In [ ]:
class Batched_Bandits:
  def __init__(self, label, K, T, M, gridType, gamma = 1):
    self.label = label
    self.algo = BaSE
    self.M = M
    self.make_grid(gridType, K, T, M)
    self.gamma = 1
      
  def make_grid(self, gridType, K, T, M):
    if gridType == 'minimax':
      a = T**(1/(2 - 2**(1-M)))
      grid = np.floor([a**(2-1/2**(i)) for i in range(M+2)])
      grid[-1] = T
      self.grid = grid
    
    elif gridType == 'geometric':
      b = T**(1/M)
      grid = np.floor([b**i for i in range(1, M+2)])
      grid[-1] = T
      self.grid = grid
    
    elif gridType == 'arithmetic':
      self.grid = np.floor(np.linspace(0, T, M+1))


In [ ]:
def BaSE(K, T, data, deltas, method, n_points, points_indices, verbose = True, adv = False):
  """ 
  Note that data contains sequence of losses and the algo is specified to work with rewards,
  so losses are converted in rewards when constructing weights.
  """

  M = method.M
  grid = np.copy(method.grid)
  gamma = method.gamma

  A = np.ones(K)
  total_pulled = 0
  t = 0
  ind = 0
  L = np.zeros(K)

  action_losses = np.zeros(K)
  algo_total_loss = 0
  algo_data_reg = np.zeros(n_points)
  algo_switch_loss = np.zeros(n_points)
  cum_reg = 0
  cum_switch = 0
  done = False
  last_played = -1
  for m in range(1, M+1):
    availableK = np.sum(A ==1)
    pullNumber = int(max(np.floor((grid[m] - grid[m-1])/availableK), 1))
    grid[m] = availableK * pullNumber + grid[m-1]
    total_pulled+= pullNumber

    for j in np.argwhere(A == 1):
      j = int(j)

      if not last_played == j:
        cum_switch += 1
        last_played = j
     
      for _ in range(pullNumber):
        if t >= T:
          # print(t, ind, m)
          break
        L[j] += 1- data[t, j]
        cum_reg += deltas[j]
        action_losses += data[t]
        algo_total_loss += data[t, j]
        if t == points_indices[ind]:
          if adv:
            best_arm_loss = np.min(action_losses)
            algo_data_reg[ind] = algo_total_loss - best_arm_loss
          else:
            algo_data_reg[ind] = cum_reg

          algo_switch_loss[ind] = cum_switch
          ind += 1
        t+= 1

    Y = L/total_pulled
    maxArm = np.max(Y)

    for j in np.argwhere(A == 1):
      j = int(j)
      if maxArm - Y[j] >= np.sqrt(gamma* np.log(T*K)/total_pulled):
        A[j] = 0

    if verbose and i%1000 == 0:
      print(t, M, np.argwhere(A == 1))
  if adv:
    best_arm_loss = np.min(action_losses)
    algo_data_reg[ind] = algo_total_loss - best_arm_loss
  else:
    algo_data_reg[ind] = cum_reg
  algo_switch_loss[ind] = cum_switch  
  return algo_data_reg, algo_switch_loss

# Experiment Implementation

In [20]:
def experiment_setup(T, K, mus, switching_cost, methods, stochastic_data, rep = 1, verbose = True, saved_points = True, n_points = 1000, adv = False):
  """
  General framework to compare different methods on the same data, for several repetitions
  """
  opt_mu = np.min(mus)
  deltas = mus - opt_mu
  
  n_methods = len(methods)
  
  if not saved_points:
     n_points = T
  points_indices = np.linspace(0, T, n_points, dtype = int)

  all_reg = np.zeros((n_points, n_methods, rep,))
  all_switch_loss = np.zeros((n_points, n_methods, rep,))

  for r in range(rep):
    print("Repetition", r)
    data = generate_data(T, K, mus, deltas, stochastic_data, adv)
    for i, method in enumerate(methods):
      
      reg, switch_loss = method.algo(K, T, data, deltas, method, n_points, points_indices, verbose, adv)
      all_reg[:, i, r]= reg
      all_switch_loss[:, i, r]= switch_loss


  avg_reg = np.zeros((n_points, n_methods,))
  avg_switch = np.zeros((n_points, n_methods,))
  avg_total_reg = np.zeros((n_points, n_methods,))

  std_reg = np.zeros((n_points, n_methods,))
  std_switch = np.zeros((n_points, n_methods,))
  std_total_reg = np.zeros((n_points, n_methods,))

  all_switch_loss = switching_cost * all_switch_loss

  for i in range(len(methods)):
    avg_reg[:, i] = np.mean(all_reg[:, i, :], axis = 1)
    avg_switch[:, i] = np.mean(all_switch_loss[:, i, :], axis = 1)
    
    std_reg[:, i] = np.std(all_reg[:, i, :], axis = 1)
    std_switch[:, i] = np.std(all_switch_loss[:, i, :], axis = 1)
    

  avg_total_reg = avg_reg + avg_switch
  std_total_reg = std_reg + std_switch

  return [avg_reg, avg_switch, avg_total_reg, std_reg, std_switch, std_total_reg, points_indices]

In [ ]:
def experiment_multiple_best(T, K, mu_base, delta, switching_cost, stochastic_data, rep = 1, verbose = True, saved_points = True, n_points = 1000, adv = False):
  """
  Specific framework to test the Tsallis Switch algo with an increasing number of optimal arms.
  """
  mus = np.ones(K) * mu_base
  n_methods = K-1
  methods = []
  if not saved_points:
     n_points = T
  points_indices = np.linspace(0, T, n_points, dtype = int)

  all_reg = np.zeros((n_points, n_methods, rep,))
  all_switch_loss = np.zeros((n_points, n_methods, rep,))

  
  for i in range(0, K-1):
    mus[i] = mus[i] - delta
    titre = str(i + 1) + " best arms"
    block_tsallis = Distrib_based(titre, K, T, method = "Tsallis", use_blocks = True, switching_cost=switching_cost)
    methods.append(block_tsallis)
    opt_mu = np.min(mus)
    deltas = mus - opt_mu
    for r in range(rep):
      data = generate_data(T, K, mus, deltas, stochastic_data, adv)

      reg, switch_loss = block_tsallis.algo(K, T, data, deltas, block_tsallis, n_points, points_indices, verbose, adv)
      all_reg[:, i, r]= reg
      all_switch_loss[:, i, r]= switch_loss


  avg_reg = np.zeros((n_points, n_methods,))
  avg_switch = np.zeros((n_points, n_methods,))
  avg_total_reg = np.zeros((n_points, n_methods,))

  std_reg = np.zeros((n_points, n_methods,))
  std_switch = np.zeros((n_points, n_methods,))
  std_total_reg = np.zeros((n_points, n_methods,))

  all_switch_loss = switching_cost * all_switch_loss

  for i in range(0, K-1):
    avg_reg[:, i] = np.mean(all_reg[:, i, :], axis = 1)
    avg_switch[:, i] = np.mean(all_switch_loss[:, i, :], axis = 1)
    
    std_reg[:, i] = np.std(all_reg[:, i, :], axis = 1)
    std_switch[:, i] = np.std(all_switch_loss[:, i, :], axis = 1)
    

  avg_total_reg = avg_reg + avg_switch
  std_total_reg = std_reg + std_switch

  return [avg_reg, avg_switch, avg_total_reg, std_reg, std_switch, std_total_reg, points_indices], methods

# Plotting function
This function creates 3 plots. One of the pseudo-regret (or regret) with switching cost, and two different scales of the pseudo-regret (or regret), switching cost and pseudo-regret (or regret) with switching cost.

In [26]:
def make_plots(methods, data, title, params, adv = False, label = ''):
  """
  Makes and saved different plots.
  methods is a list of the methods used, to extrac their labels
  title is displayed on the single plot
  adv displays "Regret" when adv == True and "Pseudo-Regret" otherwise
  params and label are only used to describe the experiment in the figure filepath.
  """
  ##  Data
  indices = data[-1]
  avg_reg = data[0]
  avg_switch = data[1]
  avg_total_reg = data[2]
  std_reg = data[3]
  std_switch = data[4]
  std_total_reg = data[5]

  ## make fig_name 
  fig_name = '_'.join(str(e) for e in params)
  now = datetime.now()
  fig_name =  label + fig_name + str(date.today()) + now.strftime("%H%M%S") 
  
  ## Plots

  ### Single Plot
  for i, method in enumerate(methods):
    lab = method.label
    plt.plot(indices, avg_total_reg[:, i], label = lab)
    plt.fill_between(indices, avg_total_reg[:, i] - std_total_reg[:, i], avg_total_reg[:, i] + std_total_reg[:, i], alpha = 0.3)
  plt.title(title)
  plt.xlabel('t')
  if adv:
     plt.ylabel('regret with swtiching cost')
  else:
    plt.ylabel('pseudo-regret with swtiching cost')
  plt.legend( bbox_to_anchor=(1.05, 1))
  plt.savefig(fig_name + "_single_plot.png", bbox_inches="tight")
  plt.close()

  ### Complete plot for double column
  fig = plt.figure(1)
  gridspec.GridSpec(5, 4)
  fig.tight_layout()
  fig.set_size_inches(w=6,h=5)

  #### small subplot 1
  plt.subplot2grid((4,4), (0,0), colspan=2, rowspan=2)
  for i, method in enumerate(methods):
    lab = method.label
    plt.plot(indices, avg_reg[:, i])
    plt.fill_between(indices, avg_reg[:, i] - std_reg[:, i], avg_reg[:, i] + std_reg[:, i], alpha = 0.3)
  if adv:
     plt.title('regret')
  else:
    plt.title('pseudo-regret')
  plt.xlabel('t')
  plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

  #### small subplot 2
  plt.subplot2grid((4,4), (0,2), colspan=2, rowspan=2)
  for i, method in enumerate(methods):
    lab = method.label
    plt.plot(indices, avg_switch[:, i])
    plt.fill_between(indices, avg_switch[:, i] - std_switch[:, i], avg_switch[:, i] + std_switch[:, i], alpha = 0.3)
  plt.title('switching Cost')
  plt.xlabel('t')
  plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

  #### large subplot
  plt.subplot2grid((4,4), (2,0), colspan=5, rowspan=4)
  for i, method in enumerate(methods):
    lab = method.label
    plt.plot(indices, avg_total_reg[:, i], label = lab)
    plt.fill_between(indices, avg_total_reg[:, i] - std_total_reg[:, i], avg_total_reg[:, i] + std_total_reg[:, i], alpha = 0.3)
  if adv:
     plt.ylabel('regret with swtiching cost')
  else:
    plt.ylabel('pseudo-regret with swtiching cost')
  plt.xlabel('t')
 
  plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
  plt.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
  plt.subplots_adjust(wspace=0.45, hspace=1.5)
  fig.savefig(fig_name + ".png", bbox_inches="tight" )
  fig.show()

  ### Complete plot for single column
  fig = plt.figure(1)
  gridspec.GridSpec(5, 4)
  fig.tight_layout()
  fig.set_size_inches(w=15,h=10)

  #### small subplot 1
  plt.subplot2grid((4,4), (0,0), colspan=2, rowspan=2)
  for i, method in enumerate(methods):
    lab = method.label
    plt.plot(indices, avg_reg[:, i])
    plt.fill_between(indices, avg_reg[:, i] - std_reg[:, i], avg_reg[:, i] + std_reg[:, i], alpha = 0.3)
  plt.title('pseudo-regret')
  plt.xlabel('t')
  plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
  if adv:
    plt.ylabel('regret')
  else:
    plt.ylabel('pseudo-regret')

  #### small subplot 2
  plt.subplot2grid((4,4), (0,2), colspan=2, rowspan=2)
  for i, method in enumerate(methods):
    lab = method.label
    plt.plot(indices, avg_switch[:, i])
    plt.fill_between(indices, avg_switch[:, i] - std_switch[:, i], avg_switch[:, i] + std_switch[:, i], alpha = 0.3)
  plt.title('switching cost')
  plt.xlabel('t')
  plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
  plt.ylabel('switching cost')

  #### large subplot
  plt.subplot2grid((4,4), (2,0), colspan=3, rowspan=2)
  for i, method in enumerate(methods):
    lab = method.label
    plt.plot(indices, avg_total_reg[:, i], label = lab)
    plt.fill_between(indices, avg_total_reg[:, i] - std_total_reg[:, i], avg_total_reg[:, i] + std_total_reg[:, i], alpha = 0.3)
  plt.xlabel('t')
  if adv:
    plt.ylabel('regret with swtiching cost')
    plt.title('regret with swtiching cost')
  else:
    plt.ylabel('pseudo-regret with swtiching cost')
    plt.title('pseudo-regret with swtiching cost')
  plt.legend( bbox_to_anchor=(1.05, 1))
 
  plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
  plt.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
  plt.subplots_adjust(wspace=0.45, hspace=1.5)
  fig.savefig(fig_name + "_big_plot.png", bbox_inches="tight" )
  fig.show()

# Experiments

In the following experiments, here are the definitions of the parameters

*   K : number of arms
*   T : number of rounds
* switching_cost : switching cost
* delta : suboptimality gap between the optimal arm and the suboptimal arms
* mu_base : expected loss of the suboptimal arm

Note that the vector mus can also be tuned manually.


* stoch_data : True to use stochastic data, False to use stochastically constrained adversarial data
* adv : True to override stoch_data to an adversarial sequence of data, False to use stoch_data

Each learner is an instance of either the Batched_Bandits class or the Distrib_based class
* M : number of blocks in the Batched Bandits algorithms


Saving plots specificities
* exp_index and params are only used to change the save path of the figures


In [22]:
### Global params

K = 8
T = 100000
rep = 10

In [ ]:
## Stochastic Losses and Easy data
# Figure 2

delta = 0.05
switching_cost = 0.025

stoch_data = True

mu_base = 0.5
mus = np.ones(K) * mu_base
mus[0] = mus[0] - delta

params = [exp_index, stoch_data, T, K, mu_base, delta, switching_cost, rep]
title = "K=" + str(K) + ", T=" + str(T) + ", $\lambda=$" + str(switching_cost) + ", $\Delta=$" + str(delta)
title = r""+title

## Make methods array
block_tsallis = Distrib_based("Tsallis with blocks", K, T, method = "Tsallis", use_blocks = True, switching_cost=switching_cost)
MAB_tsallis = Distrib_based("MAB Tsallis", K, T,method =  "Tsallis", use_blocks = False)
EXP3 = Distrib_based("EXP3", K, T, method = "EXP3", use_blocks=False)
blocks_EXP3  = Distrib_based("EXP3 with blocks", K, T, method = "EXP3", use_blocks = True, switching_cost=switching_cost, adaptive = False)

M =  int(10 *np.log(T))
base_ari = Batched_Bandits("Batched_Bandits with arithmetic blocks", K, T, M, 'arithmetic', gamma = 0.5)
base_geo = Batched_Bandits("Batched_Bandits with geometric blocks", K, T, M, 'geometric', gamma = 0.5)
methods = [block_tsallis, MAB_tsallis, EXP3, blocks_EXP3, base_ari, base_geo]

## run experiment
data2 = experiment_setup(T, K, mus, switching_cost, methods, stoch_data, rep, verbose = False, saved_points=False)
## plot
make_plots(methods, data2, title, params, "figure_stoch_easy")


In [ ]:
## Stochastic Losses and Hard data
# Figure 3

delta = 0.05
switching_cost = 1
stoch_data = True

mu_base = 0.5
mus = np.ones(K) * mu_base
mus[0] = mus[0] - delta

params = [exp_index, stoch_data, T, K, mu_base, delta, switching_cost, rep]
title = "K=" + str(K) + ", T=" + str(T) + ", $\lambda=$" + str(switching_cost) + ", $\Delta=$" + str(delta)
title = r""+title

## Make methods array
block_tsallis = Distrib_based("Tsallis with blocks", K, T, method = "Tsallis", use_blocks = True, switching_cost=switching_cost)
MAB_tsallis = Distrib_based("MAB Tsallis", K, T,method =  "Tsallis", use_blocks = False)
EXP3 = Distrib_based("EXP3", K, T, method = "EXP3", use_blocks=False)
blocks_EXP3  = Distrib_based("EXP3 with blocks", K, T, method = "EXP3", use_blocks = True, switching_cost=switching_cost, adaptive = False)

M =  int(10 *np.log(T))
base_ari = Batched_Bandits("Batched_Bandits with arithmetic blocks", K, T, M, 'arithmetic', gamma = 0.5)
base_geo = Batched_Bandits("Batched_Bandits with geometric blocks", K, T, M, 'geometric', gamma = 0.5)
methods = [block_tsallis, MAB_tsallis, EXP3, blocks_EXP3, base_ari, base_geo]

## run experiment
data3 = experiment_setup(T, K, mus, switching_cost, methods, stoch_data, rep, verbose = False, saved_points=False)
## plot
make_plots(methods, data3, title, params, "figure_stoch_hard")

In [ ]:
## Stochastic Losses and no switching cost
# Figure 4

delta = 0.2
switching_cost = 0

stoch_data = True

mu_base = 0.5
mus = np.ones(K) * mu_base
mus[0] = mus[0] - delta

params = [exp_index, stoch_data, T, K, mu_base, delta, switching_cost, rep]
title = "K=" + str(K) + ", T=" + str(T) + ", $\lambda=$" + str(switching_cost) + ", $\Delta=$" + str(delta)
title = r""+title

## Make methods array
block_tsallis = Distrib_based("Tsallis with blocks", K, T, method = "Tsallis", use_blocks = True, switching_cost=switching_cost)
MAB_tsallis = Distrib_based("MAB Tsallis", K, T,method =  "Tsallis", use_blocks = False)
EXP3 = Distrib_based("EXP3", K, T, method = "EXP3", use_blocks=False)
blocks_EXP3  = Distrib_based("EXP3 with blocks", K, T, method = "EXP3", use_blocks = True, switching_cost=switching_cost, adaptive = False)

M =  int(10 *np.log(T))
base_ari = Batched_Bandits("Batched_Bandits with arithmetic blocks", K, T, M, 'arithmetic', gamma = 0.5)
base_geo = Batched_Bandits("Batched_Bandits with geometric blocks", K, T, M, 'geometric', gamma = 0.5)

methods = [block_tsallis, MAB_tsallis, EXP3, blocks_EXP3, base_ari, base_geo]

## run experiment
data4 = experiment_setup(T, K, mus, switching_cost, methods, stoch_data, rep, verbose = False)
## plot
make_plots(methods, data4, title, params, "figure_stoch_no_switching_cost")

In [ ]:
## Stochasically Constrained Adversarial, easy data
# Figure 5

delta = 0.2
switching_cost = 0.025
stoch_data = False

mu_base = 0.5
mus = np.ones(K) * mu_base
mus[0] = mus[0] - delta


params = [stoch_data, T, K, mu_base, delta, switching_cost, rep]
title = "K=" + str(K) + ", T=" + str(T) + ", $\lambda=$" + str(switching_cost) + ", $\Delta=$" + str(delta)
title = r""+title

## Make methods array
block_tsallis = Distrib_based("Tsallis with blocks", K, T, method = "Tsallis", use_blocks = True, switching_cost=switching_cost)
MAB_tsallis = Distrib_based("MAB Tsallis", K, T,method =  "Tsallis", use_blocks = False)
EXP3 = Distrib_based("EXP3", K, T, method = "EXP3", use_blocks=False)
blocks_EXP3  = Distrib_based("EXP3 with blocks", K, T, method = "EXP3", use_blocks = True, switching_cost=switching_cost, adaptive = False)

M =  int(10 *np.log(T))
base_ari = Batched_Bandits("Batched_Bandits with arithmetic blocks", K, T, M, 'arithmetic', gamma = 0.5)
base_geo = Batched_Bandits("Batched_Bandits with geometric blocks", K, T, M, 'geometric', gamma = 0.5)
methods = [block_tsallis, MAB_tsallis, EXP3, blocks_EXP3, base_ari, base_geo]

## run experiment
data5 = experiment_setup(T, K, mus, switching_cost, methods, stoch_data, rep, verbose = False)
## plot
make_plots(methods, data5, title, params, "figure_non_stoch_easy")

In [ ]:
## Stochasically Constrained Adversarial, hard data
# Figure 6

delta = 0.05
switching_cost = 1
stoch_data = False

mu_base = 0.5
mus = np.ones(K) * mu_base
mus[0] = mus[0] - delta


params = [stoch_data, T, K, mu_base, delta, switching_cost, rep]
title = "K=" + str(K) + ", T=" + str(T) + ", $\lambda=$" + str(switching_cost) + ", $\Delta=$" + str(delta)
title = r""+title

## Make methods array
block_tsallis = Distrib_based("Tsallis with blocks", K, T, method = "Tsallis", use_blocks = True, switching_cost=switching_cost)
MAB_tsallis = Distrib_based("MAB Tsallis", K, T,method =  "Tsallis", use_blocks = False)
EXP3 = Distrib_based("EXP3", K, T, method = "EXP3", use_blocks=False)
blocks_EXP3  = Distrib_based("EXP3 with blocks", K, T, method = "EXP3", use_blocks = True, switching_cost=switching_cost, adaptive = False)

M =  int(10 *np.log(T))
base_ari = Batched_Bandits("Batched_Bandits with arithmetic blocks", K, T, M, 'arithmetic', gamma = 0.5)
base_geo = Batched_Bandits("Batched_Bandits with geometric blocks", K, T, M, 'geometric', gamma = 0.5)

methods = [block_tsallis, MAB_tsallis, EXP3, blocks_EXP3, base_ari, base_geo]

## run experiment
data6 = experiment_setup(T, K, mus, switching_cost, methods, stoch_data, rep, verbose = False)
## plot
make_plots(methods, data6, title, params, "figure_non_stoch_hard")

In [ ]:
## Adversarial sequence
# Experiment Figure 7

delta = 0.05
switching_cost = 1
stoch_data = True
adv = True

mu_base = 0.5
mus = np.ones(K) * mu_base
mus[0] = mus[0] - delta


params = [stoch_data, T, K, mu_base, delta, switching_cost, rep]
title = 'Regret against an adversarial sequence'
title = r""+title

## Make methods array
block_tsallis = Distrib_based("Tsallis with blocks", K, T, method = "Tsallis", use_blocks = True, switching_cost=switching_cost)
MAB_tsallis = Distrib_based("MAB Tsallis", K, T,method =  "Tsallis", use_blocks = False)
EXP3 = Distrib_based("EXP3", K, T, method = "EXP3", use_blocks=False)
blocks_EXP3  = Distrib_based("EXP3 with blocks", K, T, method = "EXP3", use_blocks = True, switching_cost=switching_cost, adaptive = False)

M =  int(10 *np.log(T))
base_ari = Batched_Bandits("Batched_Bandits with arithmetic blocks", K, T, M, 'arithmetic', gamma = 0.5)
base_geo = Batched_Bandits("Batched_Bandits with geometric blocks", K, T, M, 'geometric', gamma = 0.5)

methods = [block_tsallis, MAB_tsallis, EXP3, blocks_EXP3, base_ari, base_geo]


## run experiment
data7 = experiment_setup(T, K, mus, switching_cost, methods, stoch_data, rep, verbose = False, adv = adv, n_points = T)
## plot
make_plots(methods, data7, title, params, label = "adv_example", adv = adv)

In [ ]:
## Comparing several number of best arms with stochstic data
# Experiment Figure 8

delta = 0.2
switching_cost = 1
stoch_data = True
adv = False
mu_base = 0.5

title = 'Multiple Occurances of the Best Arm'
title = r""+title
params = "Stoch_data_multi_best"
total_data = [] 

data8, methods = experiment_multiple_best(T, K, mu_base, delta, switching_cost, stoch_data, rep, verbose = False, adv = adv, n_points = T)
 
## plot
make_plots(methods, data8, title, params, label = "multi_best_stoch", adv = adv)


In [ ]:
## Comparing several number of best arms with non stochastic data
# Experiment Figure 9

delta = 0.2
switching_cost = 1
stoch_data = False
adv = False
mu_base = 0.5

title = 'Multiple Occurances of the Best Arm with Stochastically Constrained Adversarial Regime'
title = r""+title
params = "non_stoch_data_multi_best"
total_data = [] 

data9, methods = experiment_multiple_best(T, K, mu_base, delta, switching_cost, stoch_data, rep, verbose = False, adv = adv, n_points = T)
 
## plot
make_plots(methods, data9, title, params, label = "non_stoch_multi_best", adv = adv)
